<div id="reminder" style="border-radius: 5px; background-color:#f5f5f5; padding: 15px 5px; " >
<p>Use this notebook to follow along with the lab tutorial.</p>
</div>

# <font color="blue">Lesson 2 - Data Preparation and Retrieval</font>

## Section 1. Retrieving Data from URL, SQL Server, and Azure Blob Storage

### Read Data from URL

In [1]:
import pandas as pd
import io
import requests
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
s = requests.get(url).content
c = pd.read_csv(io.StringIO(s.decode('utf-8')))
c.columns = ["age", "workclass", "fnlwgt", "education", "education_num", \
                    "marital-status", "occupation", "relationship", "race", "sex", \
                    "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"]
c.head()

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


### Read Data from SQL Server (DO NOT Run in CLASS)

In [ ]:
import pyodbc
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=<server URL>; DATABASE=<database name>; UID=<user id>;PWD= <pwd>')
cursor = cnxn.cursor()
cursor.execute("SELECT WORK_ORDER.TYPE,WORK_ORDER.STATUS, WORK_ORDER.BASE_ID, WORK_ORDER.LOT_ID FROM WORK_ORDER")
for row in cursor.fetchall():
    print row
cursor.close()
cnxn.close()

### Read Azure Blob Storage

In [14]:
from azure.storage.blob import BlockBlobService
blob_account_name = "uwcoursestorage"
import getpass
storage_key = getpass.getpass("Please input the storage account key:")
block_blob_service = BlockBlobService(account_name=blob_account_name, account_key=storage_key)
generator = block_blob_service.list_blobs('testdata')
print("Blob names in container are:")
for blob in generator:
    print(blob.name)
block_blob_service.get_blob_to_path('testdata', 'RetailChurnData.csv', 'RetailChurnData.csv')

df = pd.read_csv("RetailChurnData.csv", header=0)
df.head()

Please input the storage account key:········
Blob names in container are:
RetailChurnData.csv


,UserId,Age,Address,Gender,UserType,Column 0,TransactionId,Timestamp,ItemId,Quantity,Value,Location,ProductCategory
0,2105345,D,F,Unknown,Unknown,220245,1215553,12/31/2000 12:00:00 AM,4710040000000,1,149,Unknown,Unknown
1,2105345,D,F,Unknown,Unknown,220246,1216545,12/31/2000 12:00:00 AM,4711090000000,1,179,Unknown,Unknown
2,2105345,D,F,Unknown,Unknown,220247,1216590,12/31/2000 12:00:00 AM,9556000000000,1,28,Unknown,Unknown
3,2105345,D,F,Unknown,Unknown,220248,1217249,12/31/2000 12:00:00 AM,4711800000000,1,199,Unknown,Unknown
4,2105345,D,F,Unknown,Unknown,220249,1217259,12/31/2000 12:00:00 AM,4710030000000,1,139,Unknown,Unknown


### Concatenate Data Frames
Appending to the same column using the `concat` function.

In [2]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'], \
                    'B': ['B0', 'B1', 'B2', 'B3'], \
                    'C': ['C0', 'C1', 'C2', 'C3'], \
                    'D': ['D0', 'D1', 'D2', 'D3']}, \
                   index=[0, 1, 2, 3])

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'], \
                    'B': ['B4', 'B5', 'B6', 'B7'], \
                    'C': ['C4', 'C5', 'C6', 'C7'], \
                    'D': ['D4', 'D5', 'D6', 'D7']},\
                   index=[4, 5, 6, 7]) 

frames = [df1, df2]

result = pd.concat(frames)
result.head(10)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


### Join two Data Frames
Using the `merge` function to join two dataframes so that they are one dtataframe with both sets of columns

In [3]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'], \
                     'A': ['A0', 'A1', 'A2', 'A3'], \
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'], \
                      'C': ['C0', 'C1', 'C2', 'C3'], \
                      'D': ['D0', 'D1', 'D2', 'D3']})


result = pd.merge(left, right, on='key')
result.head()

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


#### Merge is an inner join by default.
left has K2 but right1 does not have K2, when we join we will drop row K2.

In [4]:
right1 = pd.DataFrame({'key': ['K0', 'K1', 'K3'], \
                      'C': ['C0', 'C1', 'C3'], \
                      'D': ['D0', 'D1', 'D3']})
result1 = pd.merge(left, right1, on='key') # Inner join by default
result1.head()

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K3,A3,B3,C3,D3


#### Merge can be left outer join by using the "how" parameter.
Recall right1 does not have K2, what do you expect as the result2? result3?

In [5]:
result2 = pd.merge(left, right1, how='left', on='key') # left outer join
result2.head()

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,NaN,NaN
3,K3,A3,B3,C3,D3


In [6]:
result3 = pd.merge(left, right1, how='right', on='key') # right outer join
result3.head()

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K3,A3,B3,C3,D3


#### Merge can also do a full outer join by "how=outer"
left1 does not have K0, what do you expect as the result4?

In [7]:
left1 = pd.DataFrame({'key': ['K1', 'K2', 'K3'], \
                     'A': ['A1', 'A2', 'A3'], \
                     'B': ['B1', 'B2', 'B3']})
result4 = pd.merge(left1, right1, how='outer', on='key') # outer join
result4.head()

,key,A,B,C,D
0,K1,A1,B1,C1,D1
1,K2,A2,B2,NaN,NaN
2,K3,A3,B3,C3,D3
3,K0,NaN,NaN,C0,D0


## Section 2. Data Exploration

### Read Data from Local CSV File

In [ ]:
file = "Bank Data.csv"
import pandas as pd
df = pd.read_csv(file, header=0)
print(df.head())
df.describe()

### Get the General Description of the Data

In [ ]:
# If you get an error in the next cell block, uncomment the following line to install pandas_profiling
# !pip install pandas_profiling

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(df)

## Section 3. Handle Missing Values and Outliers

### Introduce Missing Values

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
%matplotlib inline
# import some data to play with
iris = datasets.load_iris()
X = pd.DataFrame(iris.data)
X.columns = ["SEPAL_LENGTH", "SEPAL_WIDTH", "PETAL_LENGTH", "PETAL_WIDTH"]
X_incomplete = X.copy()
X_incomplete.loc[150] = [np.NaN, np.NaN, np.NaN, np.NaN]

X_incomplete.tail()

### Remove Rows with Missing Values

In [ ]:
X_incomplete1 = X_incomplete.copy()
X_incomplete1.dropna(inplace=True)
X_incomplete1.tail()

### Imputation: Replace Missing Values with Mean

In [ ]:
X_incomplete1 = X_incomplete.copy()
X_incomplete1.fillna(X_incomplete1.mean(), inplace=True)
# count the number of NaN values in each column
X_incomplete1.tail()

### Imputation: Use Scikit-Learn Preprocessing
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html

- Can use mean, median, most frequent for the "strategy".
- This method can also impute rowwise as well as columnwise (cols: axis 0 vs. rows: axis 1).
- Works for numeric and categorical measures because of "most frequent" strategy.

In [ ]:
X_incomplete1 = X_incomplete.copy()
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy="median") #default is mean
transformed_values = pd.DataFrame(imputer.fit_transform(X_incomplete1))
transformed_values.tail()

## Handle Outliers

### Winsorize

In [ ]:
X = [92,19,101,58,1053,91,26,78,10,13,-40,101,86,85,15,89,89,28,-5,41]
print("mean=%.2f"%np.mean(X))
from scipy.stats import mstats
X1 = mstats.winsorize(X, limits=0.05, inplace=True) #It is thresholding at 5th and 95th percentiles
print(X1)
print("After winsorizing, mean=%.2f"%np.mean(X1))

### Statistics Robust to Outliers
Impact of outliers on statistics and how we can mitigate

In [ ]:
mu, sigma = 0, 2 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
import matplotlib.pyplot as plt
count, bins, ignored = plt.hist(s, 30, normed=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ), linewidth=2, color='r')
plt.show()
print("mean=%.2f, sd=%.2f"%(np.mean(s), np.std(s)))

# Introduce an outlier
s[983] = 200
print("with outlier, mean=%.2f, sd=%.2f"%(np.mean(s), np.std(s)))

from statsmodels import robust
print("with outlier, median=%.2f, sd=%.2f"%(np.median(s), 1.4826*robust.mad(s)))

In [ ]:
from statsmodels import robust

## Section 4. Scaling, Binning, and Data Transformation

### Scale to between 0 and 1

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
data_minmax = df.copy()
data_minmax.iloc[:,[0,3,5]] = min_max_scaler.fit_transform(data_minmax.iloc[:,[0,3,5]])
data_minmax.describe()

### z-transaction
Shifting the decimal places

In [ ]:
from sklearn import preprocessing
data_z = df.copy()
data_z.iloc[:,[0,3,5]] = preprocessing.scale(data_z.iloc[:,[0,3,5]])
data_z.describe()

### Binning the Continuous Variable into Bins
Using the Adult Income Data as an example. Data has been read into data frame c in the beginning of this Jupyter Notebook.

In [ ]:
c.head()

In [ ]:
import numpy as np
bins = np.linspace(15,70,12) #Generating a sequence with 12 numbers, step size 5
print(bins) #print the bins definition
age_binned = pd.cut(c.iloc[:,0], bins, right=True, labels=range(11)) # put the age into bins
print("Original value=%d, bin label=%d"%(c.iloc[1,0], age_binned[1]))
print("Original value=%d, bin label=%d"%(c.iloc[2,0], age_binned[2]))

## Handle Time Stamps

### TimeStamps in Strings

In [ ]:
datestring = "2017/11/06 19:00:13" # A Monday
from datetime import datetime

datetime_object = datetime.strptime(datestring, '%Y/%m/%d %H:%M:%S')
print("Year=%d"%datetime_object.year)
print("Month=%d"%datetime_object.month)
print("Day of Month=%d"%datetime_object.day)
print("Weekday=%d"%datetime_object.weekday())
print("Week Number=%s"%datetime_object.strftime("%U"))
print("Hour=%d"%datetime_object.hour)
print("Minute=%d"%datetime_object.minute)

### Handle Unix Time Stamp

In [ ]:
ts = 1352068320
datetime_object = datetime.fromtimestamp(ts)
print(datetime_object)
print("Year=%d"%datetime_object.year)
print("Month=%d"%datetime_object.month)
print("Day of Month=%d"%datetime_object.day)
print("Weekday=%d"%datetime_object.weekday())
print("Week Number=%s"%datetime_object.strftime("%U"))
print("Hour=%d"%datetime_object.hour)
print("Minute=%d"%datetime_object.minute)

<div id="reminder" style="border-radius: 5px; background-color:#f5f5f5; padding: 15px 5px; " >
<p>For additional practice, please see the Workshop notebooks.</p>
</div>